# YOLO

In [ ]:
!pip install ultralytics

In [2]:
from ultralytics import YOLO
model = YOLO("https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt")
results = model("/content/Parstech-ai.jpg")
for result in results:
    result.save(filename="result.jpg")


100%|██████████| 6.25M/6.25M [00:00<00:00, 126MB/s]



image 1/1 /content/Parstech-ai.jpg: 480x640 8 persons, 1 handbag, 1 tie, 44.6ms
Speed: 11.0ms preprocess, 44.6ms inference, 1008.7ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
model.export(format="onnx")

In [4]:
onnx_model = YOLO("/content/weights/yolov8n.onnx")
results = onnx_model("/content/Parstech-ai.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/weights/yolov8n.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 191.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 320.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 11.0s, installed 1 package: ['onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


image 1/1 /content/Parstech-ai.jpg: 640x640 7 persons, 14.1ms
Speed: 2.1ms preprocess, 14.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


# sklearn

In [43]:
!pip install skl2onnx

In [44]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

iris = load_iris()
X,y = iris.data , iris.target
X = X.astype(np.float32)
X_train , X_test,y_train,y_test = train_test_split(X,y)
# model = RandomForestClassifier()
model = MLPClassifier(random_state=1, max_iter=300)
model.fit(X_train,y_train)

MLPClassifier(max_iter=300, random_state=1)

In [45]:
import onnxruntime as ort
import skl2onnx
onnx_model = skl2onnx.to_onnx(model, X[0])
with open("skl2onnxmodel.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


onnx_session = ort.InferenceSession(
    "skl2onnxmodel.onnx",
    providers=["CPUExecutionProvider"]
)

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
start_time = time.time()
pred = onnx_session.run([output_name], {input_name: X_test[0]})
end_time = time.time()
pred
end_time - start_time

0.00019121170043945312

# torch

In [18]:
!pip install onnx
!pip install onnxscript
!pip install onnxruntime

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [20]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

In [21]:
onnx_program.save("my_image_classifier.onnx")

In [22]:
import onnx
onnx_model= onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

In [23]:
import onnxruntime as ort
onnx_session = ort.InferenceSession(
    "my_image_classifier.onnx",
    providers=["CPUExecutionProvider"]
)
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
input_name,output_name

onnx_input = torch_input.detach().cpu().numpy()
pred = onnx_session.run([output_name],{input_name:onnx_input})
pred

[array([[  -0.031855,   -0.070876,   -0.011677,   -0.052645,    0.063938,   0.0074441,    -0.12582,     0.14289,    0.092258,    0.028948]], dtype=float32)]

In [24]:
onnxruntime_outputs = pred

In [25]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[  -0.031855,   -0.070876,   -0.011677,   -0.052645,    0.063938,   0.0074441,    -0.12582,     0.14289,    0.092258,    0.028948]], dtype=float32)]


# TensorFlow

In [26]:
!pip install tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 11.6 MB/s eta 0:00:00


In [30]:
import numpy as np
import tensorflow as tf
from PIL import Image
import onnx
import onnxruntime as ort
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions
import tf2onnx

model = ResNet50(weights="imagenet", include_top=True)


In [31]:
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    [tf.TensorSpec(
        model.inputs[0].shape,
        dtype=model.inputs[0].dtype,
        name=model.inputs[0].name
    )]
)

In [32]:
onnx.save(onnx_model, "ResNet50.onnx")

In [36]:
onnx_session = ort.InferenceSession(
    "ResNet50.onnx",
    providers=["CPUExecutionProvider"]
)

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

image = Image.open("/content/master_groph_11-16_3.jpg")
image = image.resize((224, 224))
image = np.array(image).astype(np.float32)
image = np.expand_dims(image, axis=0)
onnx_input = image
pred = onnx_session.run([output_name], {input_name: onnx_input})


In [41]:
image_path = "/content/master_groph_11-16_3.jpg"
onnx_model_path = "ResNet50.onnx"
def prepare_image(image_path):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image = np.array(image).astype(np.float32)
    image = np.expand_dims(image, axis=0)
    return image
onnx_input = prepare_image(image_path)
cpu_start_time = time.time()
cpu_session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])
input_name = cpu_session.get_inputs()[0].name
output_name = cpu_session.get_outputs()[0].name
cpu_pred = cpu_session.run([output_name], {input_name: onnx_input})
cpu_decoded = decode_predictions(cpu_pred[0], top=5)[0]
cpu_end_time = time.time()
cpu_execution_time = cpu_end_time - cpu_start_time

gpu_start_time = time.time()
gpu_session = ort.InferenceSession(onnx_model_path, providers=["CUDAExecutionProvider"])
input_name = gpu_session.get_inputs()[0].name
output_name = gpu_session.get_outputs()[0].name
gpu_pred = gpu_session.run([output_name], {input_name: onnx_input})
gpu_decoded = decode_predictions(gpu_pred[0], top=5)[0]
gpu_end_time = time.time()
gpu_execution_time = gpu_end_time - gpu_start_time

print(f"CPU execution time: {cpu_execution_time:.4f} seconds")
print(f"GPU execution time: {gpu_execution_time:.4f} seconds")


CPU execution time: 0.2101 seconds
GPU execution time: 0.7516 seconds


In [37]:
decode_predictions(pred[0])

35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[[('n02132136', 'brown_bear', 0.7363602),
  ('n02447366', 'badger', 0.13681011),
  ('n02443114', 'polecat', 0.023592463),
  ('n01883070', 'wombat', 0.017849606),
  ('n02133161', 'American_black_bear', 0.016436473)]]